# Observer Pattern

The Observer pattern defines a one-to-many dependency between objects so that when one object changes state, all its dependents are notified and updated automatically.

## Problem

You need to be notified when the state of another object changes, but you don't want to continuously check for changes or tightly couple the objects involved.

## Solution

Define a subscription mechanism to notify multiple objects about events that happen to the object they're observing. The subject maintains a list of its dependents (observers) and notifies them automatically when its state changes.

In [1]:
from abc import ABC, abstractmethod
from typing import List


# Observer interface
class Observer(ABC):
    @abstractmethod
    def update(self, subject) -> None:
        pass


# Subject interface
class Subject(ABC):
    @abstractmethod
    def attach(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def detach(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

In [2]:
# Concrete Subject - Weather Station
class WeatherStation(Subject):
    def __init__(self):
        self._observers: List[Observer] = []
        self._temperature = 0
        self._humidity = 0
        self._pressure = 0

    def attach(self, observer: Observer) -> None:
        if observer not in self._observers:
            self._observers.append(observer)
            print(f"Subject: Attached an observer.")

    def detach(self, observer: Observer) -> None:
        self._observers.remove(observer)
        print(f"Subject: Detached an observer.")

    def notify(self) -> None:
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self)

    def set_measurements(self, temperature, humidity, pressure):
        print("\nWeather Station: Setting new measurements...")
        self._temperature = temperature
        self._humidity = humidity
        self._pressure = pressure
        self.notify()

    @property
    def temperature(self):
        return self._temperature

    @property
    def humidity(self):
        return self._humidity

    @property
    def pressure(self):
        return self._pressure

In [3]:
# Concrete Observers
class CurrentConditionsDisplay(Observer):
    def update(self, weather_station):
        print(f"Current Conditions: {weather_station.temperature}°C, {weather_station.humidity}% humidity")


class StatisticsDisplay(Observer):
    def __init__(self):
        self.temperatures = []

    def update(self, weather_station):
        self.temperatures.append(weather_station.temperature)
        avg = sum(self.temperatures) / len(self.temperatures)
        print(
            f"Statistics: Avg temperature = {avg:.1f}°C, "
            f"Max = {max(self.temperatures)}°C, "
            f"Min = {min(self.temperatures)}°C"
        )


class ForecastDisplay(Observer):
    def __init__(self):
        self.last_pressure = 0

    def update(self, weather_station):
        if self.last_pressure == 0:
            print("Forecast: Waiting for more data...")
        elif weather_station.pressure > self.last_pressure:
            print("Forecast: Improving weather on the way!")
        elif weather_station.pressure < self.last_pressure:
            print("Forecast: Watch out for cooler, rainy weather")
        else:
            print("Forecast: More of the same")

        self.last_pressure = weather_station.pressure

In [4]:
# Client code
# Create the subject
weather_station = WeatherStation()

# Create observers
current_display = CurrentConditionsDisplay()
statistics_display = StatisticsDisplay()
forecast_display = ForecastDisplay()

# Register observers
weather_station.attach(current_display)
weather_station.attach(statistics_display)
weather_station.attach(forecast_display)

# Simulate weather changes
weather_station.set_measurements(25, 65, 1013)
weather_station.set_measurements(26, 70, 1014)
weather_station.set_measurements(24, 90, 1010)

# Unregister an observer
weather_station.detach(forecast_display)

# One more measurement
weather_station.set_measurements(22, 80, 1012)

Subject: Attached an observer.
Subject: Attached an observer.
Subject: Attached an observer.

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 25°C, 65% humidity
Statistics: Avg temperature = 25.0°C, Max = 25°C, Min = 25°C
Forecast: Waiting for more data...

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 26°C, 70% humidity
Statistics: Avg temperature = 25.5°C, Max = 26°C, Min = 25°C
Forecast: Improving weather on the way!

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 24°C, 90% humidity
Statistics: Avg temperature = 25.0°C, Max = 26°C, Min = 24°C
Forecast: Watch out for cooler, rainy weather
Subject: Detached an observer.

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 22°C, 80% humidity
Statistics: Avg temperature = 24.2°C, Max = 26°C, Min = 22°C


## Benefits

* **Loose Coupling**: Subjects don't need to know anything about their observers
* **Open/Closed Principle**: You can add new observer classes without changing the subject
* **Dynamic Relationships**: You can establish relationships between objects at runtime
* **Broadcast Communication**: A subject can notify all interested objects automatically
* **Event-Driven Architecture**: Supports building event-driven systems and reactive programming